In [12]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait # no hara operaciones hasta q se carguen los elementos de la página
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [16]:
# Opciones de navegacion

options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
options.add_argument("--disable-extensions")

# Inicialimos el navegador 
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get("https://es.investing.com/")



In [33]:
# Haremos clicks en los diferentes botones para llegar a los futuros de azúcar que se transan en Londrés:
# futuros azúcar N° 5 Londres
WebDriverWait(driver,5)\
    .until(EC.element_to_be_clickable((By.CSS_SELECTOR,"class#searchText")))\
    .send_keys("futuros azúcar N° 5 Londres")

TimeoutException: Message: 
Stacktrace:
Backtrace:
	Ordinal0 [0x00A478B3+2193587]
	Ordinal0 [0x009E0681+1771137]
	Ordinal0 [0x008F41A8+803240]
	Ordinal0 [0x009224A0+992416]
	Ordinal0 [0x0092273B+993083]
	Ordinal0 [0x0094F7C2+1177538]
	Ordinal0 [0x0093D7F4+1103860]
	Ordinal0 [0x0094DAE2+1170146]
	Ordinal0 [0x0093D5C6+1103302]
	Ordinal0 [0x009177E0+948192]
	Ordinal0 [0x009186E6+952038]
	GetHandleVerifier [0x00CF0CB2+2738370]
	GetHandleVerifier [0x00CE21B8+2678216]
	GetHandleVerifier [0x00AD17AA+512954]
	GetHandleVerifier [0x00AD0856+509030]
	Ordinal0 [0x009E743B+1799227]
	Ordinal0 [0x009EBB68+1817448]
	Ordinal0 [0x009EBC55+1817685]
	Ordinal0 [0x009F5230+1856048]
	BaseThreadInitThunk [0x774DFA29+25]
	RtlGetAppContainerNamedObjectPath [0x77907A9E+286]
	RtlGetAppContainerNamedObjectPath [0x77907A6E+238]


In [2]:
#Funcion para capturar los objetos por etiqueta y clase
def buscar_tag_por_etiqueta_clase(html, etiqueta, clase):
    soup = BeautifulSoup(html, "html.parser")
    return soup.find_all(etiqueta, class_= re.compile(clase +"$"))
 

In [3]:
class Producto:
    pass

In [4]:
#usaremos request para consultar por una dirección url y obtener el contenido, en este caso leer productos de supermecado lider
ruta = "https://www.lider.cl/supermercado/search?Ntt=azucar&ost=azucar"
sitio = requests.get(ruta)

In [5]:
#Buscamos nuestros tag que contienen los productos del carro
listado_items = buscar_tag_por_etiqueta_clase(sitio.content, "div", "product-item-box")

In [6]:
#Ahora vamos a recorrer cada producto del carro, según la lista que encontramos en el paso anterior. Generamos una lista de productos donde 
#guardaremos nuestros objeto Producto, según los valores que rescatamos dentro del ciclo for

productos = []
for i in listado_items:
    producto = Producto()

    if(i["prod-number"] != ""):
        producto.producto_codigo = i["prod-number"]
        
    if (i.find('span', class_="product-description js-ellipsis", recursive=True) != []):
        producto.nombre = i.find('span', class_="product-description js-ellipsis", recursive=True).text
    
    if (i.find('span', class_="product-name", recursive=True) != []):
        producto.marca = i.find('span', class_="product-name", recursive=True).text
      
    if (i.find('span', class_="product-attribute", recursive=True) != []):
        producto.formato = i.find('span', class_="product-attribute", recursive=True).text
        
    if (i.find('span', class_="price-sell", recursive=True) != []):
        producto.valor = i.find('span', class_="price-sell", recursive=True).text
    
    if (i.find('div', class_="product-legend hidden", recursive=True) != []):
        producto.vigencia = i.find('div', class_="product-legend hidden", recursive=True).text
   
    productos.append(producto)


In [7]:
#Creamos un dataframe a partir de nuestra lista de productos capturados en el paso anterior
df = pd.DataFrame.from_records([t.__dict__ for t in productos ])
df.head(200)

,producto_codigo,nombre,marca,formato,valor,vigencia
0,2437,Azúcar Blanca,Iansa,1 Kg,$1.200,Vigencia del 03-8-16 al 04-8-16. Stock: 200 Un.
1,3895,Bebida Sin Azúcar,Coca-Cola,"2,5 L",$2.110,Vigencia del 03-8-16 al 04-8-16. Stock: 200 Un.
2,579038,Azúcar Blanca Granulada,Lider,1 kg,$990,Vigencia del 03-8-16 al 04-8-16. Stock: 200 Un.
3,5609,Bebida Sin Azúcar,Coca-Cola,3 L,$2.520,Vigencia del 03-8-16 al 04-8-16. Stock: 200 Un.
4,3879,Bebida Sin Azúcar,Coca-Cola,"1,5 L",$1.690,Vigencia del 03-8-16 al 04-8-16. Stock: 200 Un.
5,8931,Azúcar Blanca,Iansa,5 Kg,$5.990,Vigencia del 03-8-16 al 04-8-16. Stock: 200 Un.
6,292735,Bebida Sin Azúcar,Coca-Cola,"4 de 1,5 L c/u",$5.720,Vigencia del 03-8-16 al 04-8-16. Stock: 200 Un.
7,11295,Néctar de Naranja Light 0% Azúcar Añadida,Watt's,"1,5 L",$1.240,Vigencia del 03-8-16 al 04-8-16. Stock: 200 Un.
8,2051,Merluza Filetes Con Piel,Lider,500 g,$3.990,Vigencia del 03-8-16 al 04-8-16. Stock: 200 Un.
9,914895,Cereal Sin Azúcar Añadida,Chocapic,720 g,$4.150,Vigencia del 03-8-16 al 04-8-16. Stock: 200 Un.


In [8]:
df2 =df[df.marca.isin(["Lider","Iansa"])]
df3 =df2[df2["nombre"]!="Merluza Filetes Con Piel"]

In [10]:
# Ahora ordenamos el DataFrame por marca:
df3.sort_values("marca")

,producto_codigo,nombre,marca,formato,valor,vigencia
0,2437,Azúcar Blanca,Iansa,1 Kg,$1.200,Vigencia del 03-8-16 al 04-8-16. Stock: 200 Un.
5,8931,Azúcar Blanca,Iansa,5 Kg,$5.990,Vigencia del 03-8-16 al 04-8-16. Stock: 200 Un.
12,674484,Azúcar Rubia,Iansa,1 Kg,$2.000,Vigencia del 03-8-16 al 04-8-16. Stock: 200 Un.
22,2439,Azúcar Rubia,Iansa,500 g,$1.150,Vigencia del 03-8-16 al 04-8-16. Stock: 200 Un.
33,2435,Azúcar Flor,Iansa,500 g,$1.090,Vigencia del 03-8-16 al 04-8-16. Stock: 200 Un.
2,579038,Azúcar Blanca Granulada,Lider,1 kg,$990,Vigencia del 03-8-16 al 04-8-16. Stock: 200 Un.
17,6433,Azúcar Granulada,Lider,5 kg,$5.310,Vigencia del 03-8-16 al 04-8-16. Stock: 200 Un.


In [11]:
file_name ="Azucar_Super_Lider.xlsx"
df3.to_excel(file_name)
df3.to_excel('Desktop/test.xlsx', encoding='utf8')
print('Azucar_Super_Lider fue exportado a Excel')

Azucar_Super_Lider fue exportado a Excel


In [ ]:
df.to_excel('Desktop/test.xlsx', encoding='utf8')

In [ ]:
# Web Scrapping página Cencosud:

#Funcion para capturar los objetos por etiqueta y clase
def buscar_tag_por_etiqueta_clase(html, etiqueta, clase):
    soup = BeautifulSoup(html, "html.parser")
    return soup.find_all(etiqueta, class_= re.compile(clase +"$"))
 

In [ ]:
class Producto:
    pass

In [ ]:
#usaremos request para consultar por una dirección url y obtener el contenido, en este caso leer productos de supermecado Jumbo
ruta = "https://www.jumbo.cl/busqueda?ft=az%C3%BAcar"
sitio = requests.get(ruta)

In [ ]:
#Buscamos nuestros tag que contienen los productos del carro
listado_items_jumbo = buscar_tag_por_etiqueta_clase(sitio.content, "div", "product-single-price")

In [ ]:
#Ahora vamos a recorrer cada producto del carro, según la lista que encontramos en el paso anterior. Generamos una lista de productos donde 
#guardaremos nuestros objeto Producto, según los valores que rescatamos dentro del ciclo for

productos = []
for i in listado_items_jumbo:
    producto = Producto()

    if(i["prod-number"] != ""):
        producto.producto_codigo = i["prod-number"]
        
    if (i.find('span', class_="product-description js-ellipsis", recursive=True) != []):
        producto.nombre = i.find('span', class_="product-description js-ellipsis", recursive=True).text
    
    if (i.find('span', class_="product-name", recursive=True) != []):
        producto.marca = i.find('span', class_="product-name", recursive=True).text
      
    if (i.find('span', class_="product-attribute", recursive=True) != []):
        producto.formato = i.find('span', class_="product-attribute", recursive=True).text
        
    if (i.find('span', class_="price-sell", recursive=True) != []):
        producto.valor = i.find('span', class_="price-sell", recursive=True).text
    
    if (i.find('div', class_="product-legend hidden", recursive=True) != []):
        producto.vigencia = i.find('div', class_="product-legend hidden", recursive=True).text
   
    productos.append(producto)

In [ ]:
#Creamos un dataframe a partir de nuestra lista de productos capturados en el paso anterior
df = pd.DataFrame.from_records([t.__dict__ for t in productos ])
df.head(200)

In [11]:
# Web Scrapping a página de Azúca Londrés 

ruta = "https://es.investing.com/commodities/london-sugar-historical-data"
sitio = requests.get(ruta) # descargamos el contenido de la página
soup = BeautifulSoup(sitio.content,"html.parser") # lo pasamos a formato BeatifulSoup

fecha = soup.find_all("tr",class_="datatable_row_2vgJ1")
print(fecha)


[]
